In [1]:
from azure.ai.ml import MLClient, spark, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import UserIdentityConfiguration

azureml_client = MLClient.from_config(
    DefaultAzureCredential(),
)

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    workspace_name='project-group-29',
    subscription_id="21ff0fc0-dd2c-450d-93b7-96eeb3699b22",
    resource_group_name='project-group-29'
)


Found the config file in: /config.json


In [3]:
# Download the spark-nlp jar and save it locally. This needs to be done before submitting a job.
import requests
response = requests.get("https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/jars/spark-nlp-assembly-5.0.2.jar")
with open("spark-nlp-assembly-5.0.2.jar", "wb") as f:
    f.write(response.content)


KeyboardInterrupt: 

In [2]:
job1_comments = spark(
    description="westies-comments",
    code="./",
    entry={"file": "Azure_job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.3",
    },
    inputs={       
        "input_object_store_base_url": "wasbs://reddit-project@dsan6000fall2024.blob.core.windows.net/",
        "input_path": "202306-202407/comments/",
        "output_object_store_base_url": "azureml://datastores/workspaceblobstore/paths/",
        "output_path": "states/comments/",
    },
    identity=UserIdentityConfiguration(),
    args="--input_object_store_base_url ${{inputs.input_object_store_base_url}} --input_path ${{inputs.input_path}} --output_object_store_base_url ${{inputs.output_object_store_base_url}} --output_path ${{inputs.output_path}}"
)

In [17]:
job1_object = azureml_client.jobs.create_or_update(job1_comments)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/prs64-ci01/code/Users/prs64/fall-2024-project-team-29/code/Powell Work' 'https://projectgstorageef8131633.blob.core.windows.net/ab2acca4-77bf-4607-9fc6-941bb7fd6768-85muyothwijkqjxwcppomsyeon/Powell Work' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading Powell Work (612.24 MBs): 100%|██████████| 612237814/612237814 [00:01<00:00, 576585836.83it/s]




In [5]:
job2_comments = spark(
    description="states_averages_round_2",
    code="./",
    entry={"file": "Sentiment_mean_job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    inputs={
        "input_data": Input(
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/states/comments/",
            mode="direct",
        ),
    },
    jars=["spark-nlp-assembly-5.0.2.jar"],
    environment="sparknlp-python-env@latest",
    identity=UserIdentityConfiguration(),
    args="--input_data ${{inputs.input_data}}"
)


In [6]:
job2_object = azureml_client.jobs.create_or_update(job2_comments)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/prs64-ci01/code/Users/prs64/fall-2024-project-team-29/code/Powell Work' 'https://projectgstorageef8131633.blob.core.windows.net/ab2acca4-77bf-4607-9fc6-941bb7fd6768-ot8fy5s5x2lgloowvkw7evj0i2/Powell Work' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading Powell Work (1320.43 MBs): 100%|██████████| 1320427232/1320427232 [00:07<00:00, 169504198.39it/s]




In [4]:
job3_comments = spark(
    description="states_averages",
    code="./",
    entry={"file": "averages.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    inputs={
        "input_data": Input(
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/states/states_sentiment",
            mode="direct",
        ),
    },
    environment="sparknlp-python-env@latest",
    identity=UserIdentityConfiguration(),
    args="--input_data ${{inputs.input_data}}"
)


In [5]:
job3_object = azureml_client.jobs.create_or_update(job3_comments)

Uploading nlp (0.05 MBs): 100%|██████████| 46320/46320 [00:00<00:00, 450718.98it/s]




In [4]:
job4_comments = spark(
    description="states_averages",
    code="./",
    entry={"file": "Top_words_by_state.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_cores=4,
    executor_memory="7g",
    executor_instances=6,
    resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    inputs={
        "input_data": Input(
            type="uri_folder",
            path="azureml://datastores/workspaceblobstore/paths/states/states_sentiment",
            mode="direct",
        ),
    },
    environment="sparknlp-python-env@latest",
    identity=UserIdentityConfiguration(),
    args="--input_data ${{inputs.input_data}}"
)


In [5]:
job4_object = azureml_client.jobs.create_or_update(job4_comments)

Uploading nlp (0.05 MBs): 100%|██████████| 46724/46724 [00:00<00:00, 420508.58it/s]


